In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# df debe tener: log_salary y RemoteWork
d = df.dropna(subset=["log_salary", "RemoteWork"]).copy()

# ANOVA: log_salary ~ grupo
model = smf.ols("log_salary ~ C(RemoteWork)", data=d).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
